<a href="https://colab.research.google.com/github/sou-venir/Dream-nori/blob/main/%EC%BD%94%EB%9E%A9%EC%9D%84%EC%9C%84%ED%95%9C%EB%93%9C%EB%A6%BC%EB%86%80%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# [1] 필수 라이브러리 설치 (코랩 환경 전용)
!pip install flask-socketio flask-ngrok pyngrok openai python-socketio
!fuser -k 5000/tcp # 기존 포트 정리

import os, json
from flask import Flask, render_template_string
from flask_socketio import SocketIO, emit
from pyngrok import ngrok
from google.colab import userdata
import openai
from google.colab import drive
import google.generativeai as genai

# [2] 구글 드라이브 마운트
drive.mount('/content/drive')
SAVE_PATH = '/content/drive/MyDrive/ChatData'
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

# [3] 보안 및 AI 설정
gemini_model = None

try:
    NGROK_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    client = openai.OpenAI(api_key=OPENAI_API_KEY)

    try:
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
        if GEMINI_API_KEY:
            genai.configure(api_key=GEMINI_API_KEY)
            gemini_model = genai.GenerativeModel('gemini-1.5-pro-latest')
    except:
        print("⚠️ 구글 API 키가 설정되지 않았습니다. Gemini 모델은 사용할 수 없습니다.")

    ngrok.set_auth_token(NGROK_TOKEN)
except Exception as e:
    print(f"❌ 설정 확인 필요: {e}")

app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*")

DATA_FILE = os.path.join(SAVE_PATH, "save_data.json")

def save_data():
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(state, f, ensure_ascii=False, indent=4)

def load_data():
    if os.path.exists(DATA_FILE):
        try:
            with open(DATA_FILE, "r", encoding="utf-8") as f:
                return json.load(f)
        except:
            return None
    return None

# [4] 서버 상태 관리
initial_state = {
    "session_title": "드림놀이",
    "theme": {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"},
    "ai_model": "gpt-5.2",
    "admin_password": "3896",
    "is_locked": False,
    "inputs": {"user1": None, "user2": None},
    "profiles": {
        "user1": {"name": "Player 1", "bio": "", "canon": ""},
        "user2": {"name": "Player 2", "bio": "", "canon": ""}
    },
    "history": [],
    "ai_history": [],
    "summary": "",
    "prologue": "",
    "sys_prompt": "당신은 숙련된 TRPG 마스터입니다.",
    "lorebook": [],
    "examples": [{"q":"", "a":""}, {"q":"", "a":""}, {"q":"", "a":""}]
}

saved_state = load_data()
state = saved_state if saved_state else initial_state

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE, theme=state.get('theme'))

# [5] 테마 분석 로직
def analyze_theme_color(title, sys_prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{
                "role": "system",
                "content": "너는 웹 디자인 전문가야. JSON으로 답해: {\"bg\": \"색\", \"panel\": \"색\", \"accent\": \"색\"}"
            }, {
                "role": "user",
                "content": f"제목: {title}\n설정: {sys_prompt}"
            }],
            response_format={ "type": "json_object" }
        )
        palette = json.loads(response.choices[0].message.content)
        default = {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"}
        default.update(palette)
        return default
    except:
        return {"bg": "#ffffff", "panel": "#f1f3f5", "accent": "#e91e63"}


HTML_TEMPLATE = """<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>드림놀이</title>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.0.1/socket.io.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

  <style>
    :root {
      --bg: {{ theme.bg if theme else '#ffffff' }};
      --panel: {{ theme.panel if theme else '#f1f3f5' }};
      --accent: {{ theme.accent if theme else '#e91e63' }};
      --text: #000;
    }

    html, body { height: 100%; margin: 0; overflow: hidden; }
    body { font-family: 'Pretendard', sans-serif; display: flex; background: var(--bg); color: #000000 !important; }
    div, p, span, h1, h2, h3, h4, input, textarea, select, button, .bubble { color: #000000 !important; }

    #main {
      flex: 1; display: flex; flex-direction: column; height: 100vh;
      border-right: 1px solid rgba(0,0,0,0.05);
      min-width: 0;
    }
    #chat-window { flex: 1; overflow-y: auto; padding: 30px 10%; display: flex; flex-direction: column; gap: 15px; }
    #chat-content { display:flex; flex-direction:column; gap:15px; }

    #sidebar{
      width: 320px;
      height: 100vh;
      background: var(--panel);
      box-sizing: border-box;
      display: flex;
      flex-direction: column;
      overflow: hidden;
      padding: 0;
    }
    #sidebar-body{
      padding: 20px;
      overflow-y: auto;
      flex: 1;
      min-height: 0;
      display:flex;
      flex-direction:column;
      gap:12px;
    }
    #sidebar-footer{
      padding: 12px 20px 16px 20px;
      border-top: 1px solid rgba(0,0,0,0.06);
      background: var(--panel);
    }

    textarea, input, select {
      background: var(--bg) !important;
      border: 1px solid rgba(0, 0, 0, 0.1) !important;
      border-radius: 10px;
      padding: 10px;
      width: 100%;
      box-sizing: border-box;
      transition: all 0.2s ease;
      resize: none !important;
    }
    #msg-input { background: var(--panel) !important; border: 1px solid rgba(0, 0, 0, 0.15) !important; height: 80px; }
    textarea:focus, input:focus { outline: none; border-color: var(--accent) !important; box-shadow: 0 0 5px rgba(0,0,0,0.05); }

    .bubble { padding: 15px 20px; border-radius: 15px; max-width: 85%; line-height: 1.6; font-size: 14px; white-space: pre-wrap; background: rgba(0,0,0,0.03); }
    .center-ai { align-self: center; background: var(--panel) !important; border-left: 5px solid var(--accent); width: 100%; max-width: 800px; box-shadow: 0 4px 15px rgba(0,0,0,0.05); }
    .user-bubble { align-self: flex-end; border-right: 5px solid var(--accent); background: var(--bg); }

    button { cursor: pointer; border: none; border-radius: 8px; background: var(--accent); padding: 10px; font-weight: bold; transition: 0.2s; color:white !important; }
    button:hover { opacity: 0.85; }
    .btn-reset { background: #ff4444 !important; color: #ffffff !important; }

    .master-btn{
      width: 100%;
      background: transparent !important;
      color: #999 !important;
      border: 1px solid #ddd !important;
      padding: 10px !important;
      border-radius: 10px;
      font-weight: 800;
    }

    /* ===== 모달 ===== */
    #admin-modal {
      display: none;
      position: fixed;
      z-index: 10000;
      left: 0; top: 0;
      width: 100vw; height: 100vh;
      background: rgba(0, 0, 0, 0.6);
      backdrop-filter: blur(5px);
      align-items: center;
      justify-content: center;
      padding: 24px;
      box-sizing: border-box;
    }

    .modal-content {
      width: 100%;
      max-width: 1200px;
      height: min(85vh, 900px);
      background: #ffffff;
      border-radius: 16px;
      display: flex;
      flex-direction: column;
      box-shadow: 0 20px 60px rgba(0,0,0,0.3);
      overflow: hidden;
      min-height: 0;
    }

    .modal-header {
      height: 60px;
      flex: 0 0 60px;
      display: flex;
      justify-content: space-between;
      align-items: center;
      padding: 0 20px;
      background: #f8f9fa;
      border-bottom: 1px solid #eee;
      box-sizing: border-box;
    }

    .tab-group { display: flex; gap: 10px; height: 100%; align-items: center; }
    .tab-btn {
      border: none;
      background: none !important;
      padding: 0 14px;
      height: 100%;
      font-size: 14px;
      font-weight: 700;
      color: #777 !important;
      cursor: pointer;
      position: relative;
    }
    .tab-btn.active { color: var(--accent) !important; }
    .tab-btn.active::after {
      content: "";
      position: absolute;
      bottom: 0; left: 0;
      width: 100%;
      height: 3px;
      background: var(--accent);
    }

    .close-btn {
      width: 32px; height: 32px;
      border-radius: 50%;
      border: none;
      background: #eee !important;
      color: #000 !important;
      font-size: 16px;
      font-weight: 800;
      cursor: pointer;
      padding: 0;
    }

    .modal-body {
      flex: 1;
      display: flex;
      overflow: hidden;
      min-height: 0;
    }

    .tab-content {
      display: none;
      width: 100%;
      height: 100%;
      flex-direction: row;
      min-height: 0;
    }
    .tab-content.active { display: flex; }

    .editor-side {
      flex: 1.25;
      padding: 20px;
      display: flex;
      flex-direction: column;
      gap: 12px;
      overflow-y: auto;
      border-right: 1px solid #f0f0f0;
      min-height: 0;
      box-sizing: border-box;
    }
    .list-side {
      flex: 0.75;
      padding: 20px;
      background: #fafafa;
      display: flex;
      flex-direction: column;
      gap: 12px;
      overflow-y: auto;
      min-height: 0;
      box-sizing: border-box;
    }

    .editor-side label, .list-side label {
      font-size: 12px;
      font-weight: 800;
      color: #999 !important;
      text-transform: uppercase;
    }

    .save-btn {
      background: var(--accent);
      color: white !important;
      padding: 14px;
      border-radius: 10px;
      font-weight: 800;
      cursor: pointer;
      border: none;
      margin-top: 4px;
    }

    /* ✅ 남는 공간 채우는 textarea */
    .fill-textarea { flex: 1; min-height: 260px; }
    /* ✅ 줄거리는 짧게 */
    .short-textarea { flex: none; height: 120px; }

    /* 학습 데이터 (textarea로 변경됨) */
    .ex-block {
      background: #fff;
      border: 1px solid #eee;
      padding: 12px;
      border-radius: 10px;
      display: flex;
      flex-direction: column;
      gap: 8px;
    }
    .ex-block textarea {
      background: #fff !important;
      border: 1px solid #ddd !important;
      border-radius: 8px;
      padding: 10px;
      font-size: 13px;
      width: 100%;
      height: 80px; /* 적당한 높이 */
      box-sizing: border-box;
    }

    /* ✅ 태그 UI */
    #tag-container{
      display:flex;
      flex-wrap:wrap;
      gap:8px;
      padding: 10px;
      border: 1px solid rgba(0,0,0,0.12);
      border-radius: 10px;
      background: var(--bg);
      align-items:center;
      min-height: 44px;
      box-sizing: border-box;
    }
    .tag-chip{
      display:inline-flex;
      align-items:center;
      gap:6px;
      padding: 6px 10px;
      border-radius: 999px;
      background: rgba(0,0,0,0.06);
      border: 1px solid rgba(0,0,0,0.08);
      font-size: 12px;
      font-weight: 700;
      user-select:none;
    }
    .tag-chip button{
      background: transparent !important;
      border: none;
      padding: 0;
      cursor: pointer;
      color: #444 !important;
      font-weight: 900;
    }
    #tag-input{
      border:none !important;
      outline:none !important;
      background: transparent !important;
      width: 220px !important;
      min-width: 120px;
      padding: 6px 8px !important;
    }

    /* 드래그 앤 드롭 스타일 */
    .draggable-item {
        cursor: grab;
        transition: background 0.2s;
    }
    .draggable-item:active { cursor: grabbing; }
    .draggable-item.dragging { opacity: 0.5; background: #eee; }
  </style>
</head>

<body>
  <div id="main">
    <div id="chat-window"><div id="chat-content"></div></div>
    <div id="input-area" style="padding:20px; background: var(--bg);">
      <div id="status" style="font-size: 12px; margin-bottom: 5px; color: var(--accent); font-weight: bold;">대기 중</div>
      <div style="display:flex; gap:10px;">
        <textarea id="msg-input" placeholder="설정 완료 후 잠금 버튼을 눌러주세요."></textarea>
        <button onclick="send()" style="width:80px;">전송</button>
      </div>
    </div>
  </div>

  <div id="sidebar">
    <div id="sidebar-body">
      <h3>🎭 설정</h3>

      <select id="user-role" onchange="refreshUI()">
        <option value="user1">Player 1</option>
        <option value="user2">Player 2</option>
      </select>

      <input type="text" id="p-name" placeholder="이름">
      <textarea id="p-bio" style="height:120px;" placeholder="캐릭터 설정"></textarea>
      <textarea id="p-canon" style="height:80px;" placeholder="관계 설정"></textarea>

      <button onclick="saveProfile()" id="ready-btn" style="background:var(--accent); color:white !important;">
        ✅ 설정 저장 및 준비 완료
      </button>

      <div id="ready-status" style="font-size:11px; margin-top:5px; color:#666;">
        대기 중...
      </div>
    </div>

    <div id="sidebar-footer">
      <button class="master-btn" onclick="requestAdmin()">⚙️ 마스터 설정</button>
    </div>
  </div>

  <div id="admin-modal">
    <div class="modal-content">
      <div class="modal-header">
        <div class="tab-group">
          <button class="tab-btn active" onclick="openTab(event, 't-base')">⚙️ 엔진</button>
          <button class="tab-btn" onclick="openTab(event, 't-story')">🎬 서사</button>
          <button class="tab-btn" onclick="openTab(event, 't-ex')">💡 학습</button>
          <button class="tab-btn" onclick="openTab(event, 't-lore')">📚 키워드</button>
        </div>
        <button onclick="closeModal()" class="close-btn">✕</button>
      </div>

      <div class="modal-body">
        <!-- 1. 엔진 -->
        <div id="t-base" class="tab-content active">
          <div class="editor-side">
            <label>AI 모델 선택</label>
            <select id="m-ai-model">
              <option value="gpt-5.2">OpenAI GPT-5.2</option>
              <option value="gpt-4o">OpenAI GPT-4o</option>
              <option value="gemini-1.5-pro-latest">Google Gemini 1.5 Pro</option>
            </select>

            <label>시스템 프롬프트 (최대 4000자)</label>
            <textarea id="m-sys" class="fill-textarea" maxlength="4000" placeholder="AI에게 줄 지침..."></textarea>

            <button onclick="saveMaster()" class="save-btn">💾 엔진 설정 저장</button>
          </div>

          <div class="list-side">
            <label>안내</label>
            <p style="font-size:13px; color:#666;">엔진 모델과 전체적인 AI의 페르소나를 결정합니다.</p>
            <button class="btn-reset" onclick="sessionReset()" style="margin-top:auto;">⚠️ 세션 완전 초기화</button>
          </div>
        </div>

        <!-- 2. 서사 -->
        <div id="t-story" class="tab-content">
          <div class="editor-side">
            <label>🏷️ 세션 제목</label>
            <input type="text" id="m-title" placeholder="제목">

            <label>📖 프롤로그 (최대 1000자)</label>
            <textarea id="m-pro" class="fill-textarea" maxlength="1000" placeholder="이야기의 시작..."></textarea>

            <button onclick="saveMaster()" class="save-btn">💾 모든 서사 저장</button>
          </div>
          <div class="list-side">
            <label>📌 현재 상황 요약 (줄거리)</label>
            <!-- 줄거리는 짧게 -->
            <textarea id="m-sum" class="short-textarea" placeholder="지금까지의 핵심 내용... (AI가 이걸 기억합니다)"></textarea>

            <label>💡 서사 팁</label>
            <p style="font-size:13px; color:#666;">오른쪽 내용은 AI가 맥락을 파악할 때 가장 중요한 기준이야.</p>
          </div>
        </div>

        <!-- 3. 학습 (Textarea로 변경) -->
        <div id="t-ex" class="tab-content">
          <div class="editor-side">
            <label>💡 말투 학습 (예시 대화 3쌍, 각 500자)</label>

            <div class="ex-block">
              <label>Example 1</label>
              <textarea id="ex-q-0" maxlength="500" placeholder="[유저]: 안녕?"></textarea>
              <textarea id="ex-a-0" maxlength="500" placeholder="[AI]: 안녕하세요! 무엇을 도와드릴까요?"></textarea>
            </div>

            <div class="ex-block">
              <label>Example 2</label>
              <textarea id="ex-q-1" maxlength="500" placeholder="[유저]: 너는 누구야?"></textarea>
              <textarea id="ex-a-1" maxlength="500" placeholder="[AI]: 저는 당신의 모험을 돕는 가이드입니다."></textarea>
            </div>

            <div class="ex-block">
              <label>Example 3</label>
              <textarea id="ex-q-2" maxlength="500" placeholder="[유저]: 상황 설명 좀"></textarea>
              <textarea id="ex-a-2" maxlength="500" placeholder="[AI]: 현재 당신은 어두운 숲 속에 서 있습니다."></textarea>
            </div>

            <button onclick="saveExamples()" class="save-btn">💡 학습 데이터 저장</button>
          </div>

          <div class="list-side">
            <label>도움말</label>
            <p style="font-size:13px; color:#666;">유저 입력/AI 출력 “한 쌍”을 3개까지 저장해서 말투를 고정할 수 있어.</p>
          </div>
        </div>

        <!-- 4. 키워드 -->
        <div id="t-lore" class="tab-content">
          <div class="editor-side">
            <label>🔍 키워드 이름</label>
            <input type="text" id="kw-t" placeholder="이름 (예: 엘프)">

            <label>🎯 트리거 (최대 5개, 엔터/스페이스로 추가)</label>
            <div id="tag-container" onclick="focusTagInput()">
              <input type="text" id="tag-input" placeholder="입력 후 Enter/Space" />
            </div>
            <input type="hidden" id="tag-hidden" value="">
            <input type="hidden" id="kw-index" value="-1"> <!-- ✅ 수정용 인덱스 저장소 -->

            <label>📝 상세 설정 (최대 400자)</label>
            <textarea id="kw-c" class="fill-textarea" maxlength="400" placeholder="AI에게 전달할 설정 내용..."></textarea>

            <div style="display:flex; gap:5px;">
                <button onclick="addLoreWithTags()" class="save-btn" style="flex:2;">➕ 키워드 저장/수정</button>
                <button onclick="clearLoreEditor()" class="save-btn" style="flex:1; background:#888;">🧹 초기화</button>
            </div>
          </div>

          <div class="list-side">
            <label>📋 저장된 키워드 (드래그로 우선순위 변경)</label>
            <div id="lore-list" style="flex: 1; overflow-y: auto; display:flex; flex-direction:column; gap:8px;"></div>
          </div>
        </div>

      </div>
    </div>
  </div>

<script>
  const socket = io();
  let gState = null;

  /* ===== 태그 로직 ===== */
  let tags = [];

  function focusTagInput(){
    const ti = document.getElementById('tag-input');
    if(ti) ti.focus();
  }

  function syncHidden(){
    document.getElementById('tag-hidden').value = tags.join(',');
  }

  function renderTags(){
    const container = document.getElementById('tag-container');
    const input = document.getElementById('tag-input');
    if(!container || !input) return;

    [...container.querySelectorAll('.tag-chip')].forEach(el => el.remove());

    tags.forEach((t, idx) => {
      const chip = document.createElement('span');
      chip.className = 'tag-chip';
      chip.innerHTML = `<span>${t}</span>`;
      const x = document.createElement('button');
      x.type = 'button';
      x.textContent = '×';
      x.onclick = (e) => { e.stopPropagation(); removeTag(idx); };
      chip.appendChild(x);
      container.insertBefore(chip, input);
    });

    syncHidden();
  }

  function addTag(raw){
    const t = (raw || '').trim();
    if(!t) return;
    if(tags.length >= 5) return alert("트리거는 최대 5개까지만 가능해!");
    if(tags.includes(t)) return;
    tags.push(t);
    renderTags();
  }

  function removeTag(idx){
    tags.splice(idx, 1);
    renderTags();
  }

  function loadTagsFromString(s){
    tags = [];
    (s || "")
      .split(',')
      .map(x => x.trim())
      .filter(Boolean)
      .forEach(x => {
        if(!tags.includes(x)) tags.push(x);
      });
    renderTags();
  }

  document.addEventListener('keydown', (e) => {
    const ti = document.getElementById('tag-input');
    if(!ti) return;
    if(document.activeElement !== ti) return;

    if(e.key === 'Enter' || e.key === ' ' || e.key === ','){
      e.preventDefault();
      addTag(ti.value);
      ti.value = '';
      return;
    }

    if(e.key === 'Backspace' && ti.value === ''){
      if(tags.length > 0){
        tags.pop();
        renderTags();
      }
    }
  });

  function clearLoreEditor(){
      document.getElementById('kw-t').value = "";
      document.getElementById('kw-c').value = "";
      document.getElementById('kw-index').value = "-1"; // 초기화 시 인덱스 리셋
      tags = [];
      renderTags();
      document.getElementById('tag-input').value = "";
  }

  /* ===== 드래그 앤 드롭 로직 ===== */
  let dragSrcEl = null;

  function handleDragStart(e) {
    dragSrcEl = this;
    e.dataTransfer.effectAllowed = 'move';
    e.dataTransfer.setData('text/plain', this.dataset.index);
    this.classList.add('dragging');
  }

  function handleDragOver(e) {
    if (e.preventDefault) e.preventDefault();
    e.dataTransfer.dropEffect = 'move';
    return false;
  }

  function handleDragEnter(e) { this.classList.add('over'); }
  function handleDragLeave(e) { this.classList.remove('over'); }

  function handleDrop(e) {
    if (e.stopPropagation) e.stopPropagation();
    const fromIndex = parseInt(e.dataTransfer.getData('text/plain'));
    const toIndex = parseInt(this.dataset.index);

    if (fromIndex !== toIndex) {
        socket.emit('reorder_lore', { from: fromIndex, to: toIndex });
    }
    return false;
  }

  function handleDragEnd(e) {
    this.classList.remove('dragging');
    [...document.querySelectorAll('.draggable-item')].forEach(item => {
        item.classList.remove('over');
    });
  }

  /* ===== 기존 로직 ===== */
  socket.on('status_update', d => {
    const statusEl = document.getElementById('status');
    if(statusEl) {
      statusEl.innerText = d.msg;
      statusEl.style.color = d.msg.includes('❌') ? 'red' : 'var(--accent)';
    }
  });

  socket.on('initial_state', data => {
    gState = data;
    if (data.theme) {
      const root = document.documentElement.style;
      root.setProperty('--bg', data.theme.bg);
      root.setProperty('--panel', data.theme.panel);
      root.setProperty('--accent', data.theme.accent);
    }
    refreshUI();
  });

  function typeWriter(element, text, i = 0) {
    if (i === 0) {
      element.innerHTML = "";
      element.style.whiteSpace = "pre-wrap";
    }
    if (i < text.length) {
      element.textContent += text.charAt(i);
      i++;
      const win = document.getElementById('chat-window');
      win.scrollTop = win.scrollHeight;
      setTimeout(() => typeWriter(element, text, i), 35);
    } else {
      element.innerHTML = marked.parse(text);
    }
  }

  function refreshUI() {
    if(!gState) return;
    renderChat();
    renderLore();
    applyLockUI();

    const activeId = document.activeElement?.id || "";
    const role = document.getElementById('user-role').value;
    const p = gState.profiles[role];

    if(activeId !== 'p-name') document.getElementById('p-name').value = p.name || "";
    if(activeId !== 'p-bio') document.getElementById('p-bio').value = p.bio || "";
    if(activeId !== 'p-canon') document.getElementById('p-canon').value = p.canon || "";

    if(activeId !== 'm-title') document.getElementById('m-title').value = gState.session_title || "";
    if(activeId !== 'm-sys') document.getElementById('m-sys').value = gState.sys_prompt || "";
    if(activeId !== 'm-pro') document.getElementById('m-pro').value = gState.prologue || "";
    if(activeId !== 'm-sum') document.getElementById('m-sum').value = gState.summary || "";

    if(gState.examples) {
      for(let i=0; i<3; i++) {
        if(gState.examples[i]) {
          if(activeId !== `ex-q-${i}`) document.getElementById(`ex-q-${i}`).value = gState.examples[i].q || "";
          if(activeId !== `ex-a-${i}`) document.getElementById(`ex-a-${i}`).value = gState.examples[i].a || "";
        }
      }
    }
  }

  function applyLockUI() {
    if(!gState) return;
    const role = document.getElementById('user-role').value;
    const p = gState.profiles[role];
    const locked = (p.name && p.name !== "Player 1" && p.name !== "Player 2");

    document.getElementById('p-name').readOnly = locked;
    document.getElementById('p-bio').readOnly = locked;
    document.getElementById('p-canon').readOnly = locked;
    document.getElementById('ready-btn').disabled = locked;
    document.getElementById('ready-btn').innerText = locked ? "🔒 설정 고정됨" : "✅ 설정 저장 및 준비 완료";
  }

  function renderChat() {
    let h = `<div style="text-align:center; padding:20px; color:var(--accent); font-weight:bold; font-size:1.4em;">${gState.session_title}</div>`;
    h += `<div class="bubble center-ai"><b>[PROLOGUE]</b><br>${marked.parse(gState.prologue || "")}</div>`;

    const contentDiv = document.getElementById('chat-content');
    const history = gState.ai_history;
    const role = document.getElementById('user-role').value;
    const pName = gState.profiles[role].name;

    history.forEach((msg, index) => {
      const isLastMsg = (index === history.length - 1);
      const isAI = msg.startsWith("**AI**:");
      const isUser = pName && msg.includes(`**${pName}**:`);

      if (isLastMsg && isAI) {
        const bubbleId = `typing-${index}`;
        h += `<div id="${bubbleId}" class="bubble center-ai"></div>`;
        contentDiv.innerHTML = h;
        const targetElement = document.getElementById(bubbleId);
        typeWriter(targetElement, msg);
      } else {
        h += `<div class="bubble ${isUser ? 'user-bubble' : 'center-ai'}">${marked.parse(msg)}</div>`;
      }
    });

    if (history.length === 0 || !history[history.length-1].startsWith("**AI**:")) {
      contentDiv.innerHTML = h;
    }

    const win = document.getElementById('chat-window');
    win.scrollTop = win.scrollHeight;
  }

  function send() {
    const input = document.getElementById('msg-input');
    const text = input.value.trim();
    if(!text) return;
    socket.emit('client_message', { uid: document.getElementById('user-role').value, text });
    input.value = '';
  }

  function requestAdmin() {
    const pw = prompt("관리자 비밀번호:");
    if(pw) socket.emit('check_admin', { password: pw });
  }

  socket.on('admin_auth_res', d => {
    if(d.success) {
      document.getElementById('admin-modal').style.display = 'flex';
      document.querySelectorAll('.tab-content').forEach(c => c.classList.remove('active'));
      document.querySelectorAll('.tab-btn').forEach(b => b.classList.remove('active'));
      document.getElementById('t-base').classList.add('active');
      document.querySelector('.tab-btn').classList.add('active');
      refreshUI();
    } else {
      alert("비밀번호 불일치");
    }
  });

  function saveMaster() {
    const masterData = {
      title: document.getElementById('m-title').value,
      sys: document.getElementById('m-sys').value,
      pro: document.getElementById('m-pro').value,
      sum: document.getElementById('m-sum').value,
      model: document.getElementById('m-ai-model').value
    };
    socket.emit('save_master_base', masterData);
    alert("저장되었습니다!");
  }

  function saveExamples() {
    const exs = [];
    for(let i=0; i<3; i++) {
      exs.push({
        q: document.getElementById(`ex-q-${i}`).value,
        a: document.getElementById(`ex-a-${i}`).value
      });
    }
    socket.emit('save_examples', exs);
    alert("학습 데이터 저장 완료.");
  }

  function addLoreWithTags() {
    const title = document.getElementById('kw-t').value;
    const content = document.getElementById('kw-c').value;
    const triggers = document.getElementById('tag-hidden').value;
    const idx = parseInt(document.getElementById('kw-index').value);

    if(!title) return alert("키워드 이름을 입력해줘!");
    if(!triggers) return alert("트리거 태그를 하나 이상 추가해줘!");
    if(!content) return alert("설정 내용을 입력해줘!");

    socket.emit('add_lore', { title, triggers, content, priority: 0, index: idx });
    clearLoreEditor();
  }

  function renderLore() {
    const listDiv = document.getElementById('lore-list');
    if(!gState || !gState.lorebook) return;

    listDiv.innerHTML = gState.lorebook.map((l, i) => `
      <div class="draggable-item" draggable="true" data-index="${i}"
           style="padding:8px; background:rgba(0,0,0,0.03); margin-bottom:5px; border-radius:8px; display:flex; justify-content:space-between; align-items:center;">
        <span onclick="editLore(${i})" style="cursor:pointer; flex:1; font-size:13px;">
          <b>${l.title}</b> <small style="color:#666;">[${l.triggers}]</small>
        </span>
        <button onclick="socket.emit('del_lore', {index:${i}})" style="padding:2px 8px; font-size:11px; background:#ff4444; color:white !important; cursor:pointer;">삭제</button>
      </div>`).join('');

    const items = document.querySelectorAll('.draggable-item');
    items.forEach(item => {
        item.addEventListener('dragstart', handleDragStart);
        item.addEventListener('dragover', handleDragOver);
        item.addEventListener('dragenter', handleDragEnter);
        item.addEventListener('dragleave', handleDragLeave);
        item.addEventListener('drop', handleDrop);
        item.addEventListener('dragend', handleDragEnd);
    });
  }

  function editLore(idx) {
    const l = gState.lorebook[idx];
    document.getElementById('kw-t').value = l.title;
    document.getElementById('kw-c').value = l.content;
    document.getElementById('kw-index').value = idx; // 수정 모드 설정
    loadTagsFromString(l.triggers || "");
  }

  function openTab(evt, id) {
    document.querySelectorAll('.tab-content').forEach(c => c.classList.remove('active'));
    document.querySelectorAll('.tab-btn').forEach(b => b.classList.remove('active'));
    document.getElementById(id).classList.add('active');
    evt.currentTarget.classList.add('active');
    if(id === 't-lore') setTimeout(focusTagInput, 50);
  }

  function closeModal() { document.getElementById('admin-modal').style.display='none'; }

  function saveProfile() {
    const role = document.getElementById('user-role').value;
    const name = document.getElementById('p-name').value;
    if(!name || name.includes("Player")) return alert("이름을 입력해주세요!");
    if(confirm("이 설정으로 확정할까? (수정 불가)")) {
      socket.emit('update_profile', {
        uid: role,
        name: name,
        bio: document.getElementById('p-bio').value,
        canon: document.getElementById('p-canon').value
      });
    }
  }

  function sessionReset() {
    if(confirm("정말 초기화해?")) {
      const pw = prompt("관리자 비번:");
      if(pw) socket.emit('reset_session', { password: pw });
    }
  }

  document.getElementById('msg-input').addEventListener('keydown', e => {
    if(e.key==='Enter' && !e.shiftKey) { e.preventDefault(); send(); }
  });

  socket.emit('request_data');
</script>
</body>
</html>
"""

# ===== 서버 로직 =====
def broadcast_state(msg=None):
    save_data()
    emit('initial_state', state, broadcast=True)
    if msg:
        emit('status_update', {'msg': msg}, broadcast=True)

@socketio.on('request_data')
def handle_request():
    emit('initial_state', state)

@socketio.on('client_message')
def on_client_message(data):
    user_text = data.get('text', '').strip()
    uid = data.get('uid')
    if not user_text:
        return

    active_context = []
    # 위쪽(앞쪽) 우선순위로 매칭된 3개만 선정
    for l in state.get('lorebook', []):
        triggers = [t.strip() for t in l.get('triggers', '').split(',') if t.strip()]
        if any(t in user_text for t in triggers):
            active_context.append(f"[{l['title']} 설정]: {l['content']}")

    messages = [{
        "role": "system",
        "content": f"{state['sys_prompt']}\n\n[현재 줄거리 요약]: {state['summary']}\n[관련 키워드 정보]: {' '.join(active_context[:3])}"
    }]

    for ex in state.get('examples', []):
        if ex.get('q') and ex.get('a'):
            messages.append({"role": "user", "content": ex['q']})
            messages.append({"role": "assistant", "content": ex['a']})

    for h in state['ai_history'][-15:]:
        messages.append({"role": "assistant" if h.startswith("**AI**") else "user", "content": h})

    current_name = state['profiles'].get(uid, {}).get('name', 'User')
    messages.append({"role": "user", "content": f"{current_name}: {user_text}"})

    try:
        current_model = state.get("ai_model", "gpt-5.2")
        emit('status_update', {'msg': f'🤔 {current_model} 생각 중...'}, broadcast=True)

        if "gemini" in current_model.lower():
            if gemini_model is None:
                raise Exception("구글 API 키가 없어 Gemini를 쓸 수 없어!")

            full_prompt = f"{state['sys_prompt']}\n[줄거리]: {state['summary']}\n"
            full_prompt += "\n".join(state['ai_history'][-10:])
            full_prompt += f"\n{current_name}: {user_text}\nAI:"

            try:
                gen_model = genai.GenerativeModel(current_model)
                response = gen_model.generate_content(full_prompt)
            except:
                response = gemini_model.generate_content(full_prompt)
            ai_response = response.text
        else:
            response = client.chat.completions.create(model=current_model, messages=messages)
            ai_response = response.choices[0].message.content

        state["ai_history"].append(f"**{current_name}**: {user_text}")
        state["ai_history"].append(f"**AI**: {ai_response}")
        broadcast_state('✅ 완료')

    except Exception as e:
        emit('status_update', {'msg': f'❌ 에러: {str(e)}'}, broadcast=True)

@socketio.on('save_master_base')
def on_save_master(data):
    state.update({
        "session_title": data.get('title', state['session_title']),
        "sys_prompt": data.get('sys', state['sys_prompt']),
        "prologue": data.get('pro', state['prologue']),
        "summary": data.get('sum', state['summary']),
        "ai_model": data.get('model', state.get('ai_model', 'gpt-5.2'))
    })
    try:
        state["theme"] = analyze_theme_color(state["session_title"], state["sys_prompt"])
    except:
        pass
    broadcast_state("💾 설정 저장됨")

@socketio.on('save_examples')
def on_save_examples(data):
    state["examples"] = data
    broadcast_state("💡 학습 데이터 저장됨")

@socketio.on('update_profile')
def on_profile(data):
    uid = data.get('uid')
    if uid not in state["profiles"]:
        return
    state["profiles"][uid].update({
        "name": data.get('name', state["profiles"][uid]["name"]),
        "bio": data.get('bio', state["profiles"][uid]["bio"]),
        "canon": data.get('canon', state["profiles"][uid]["canon"])
    })
    broadcast_state("👤 프로필 업데이트")

# [수정된 키워드 저장 로직: 인덱스 확인 후 덮어쓰기 or 추가]
@socketio.on('add_lore')
def on_add_lore(data):
    idx = int(data.get('index', -1))
    new_item = {
        'title': data['title'],
        'triggers': data['triggers'],
        'content': data['content'],
        'priority': data.get('priority', 0)
    }

    state.setdefault("lorebook", [])

    if idx >= 0 and idx < len(state["lorebook"]):
        state["lorebook"][idx] = new_item
        broadcast_state("✏️ 키워드 수정됨")
    else:
        state["lorebook"].append(new_item)
        broadcast_state("➕ 키워드 추가됨")

@socketio.on('del_lore')
def on_del_lore(data):
    try:
        state["lorebook"].pop(data['index'])
        broadcast_state("🗑️ 키워드 삭제됨")
    except:
        pass

@socketio.on('reorder_lore')
def on_reorder_lore(data):
    try:
        l = state["lorebook"]
        f, t = data['from'], data['to']
        if 0 <= f < len(l) and 0 <= t < len(l):
            item = l.pop(f)
            l.insert(t, item)
            broadcast_state("⇅ 순서 변경됨")
    except:
        pass

@socketio.on('check_admin')
def check_admin(data):
    emit('admin_auth_res', {'success': str(data.get('password')) == str(state.get('admin_password'))})

@socketio.on('reset_session')
def on_reset_session(data):
    if str(data.get('password')) == str(state.get('admin_password')):
        state.update({"ai_history": [], "lorebook": [], "summary": "", "is_locked": False})
        broadcast_state("♻️ 초기화 완료")
    else:
        emit('status_update', {'msg': '❌ 비번 틀림'}, broadcast=True)

if __name__ == '__main__':
    try:
        import subprocess
        subprocess.run(["pkill", "-9", "ngrok"])
        ngrok.kill()

        public_url = ngrok.connect(5000).public_url
        print(f"\n🚀 서버 시작! 접속 주소: {public_url}")
        socketio.run(app, host='0.0.0.0', port=5000, allow_unsafe_werkzeug=True)
    except Exception as e:
        print(f"❌ 실행 오류: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



🚀 서버 시작! 접속 주소: https://unadhesively-prolix-deb.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 07:22:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 07:22:09] "GET /socket.io/?EIO=4&transport=polling&t=PjFF9Lt HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 07:22:09] "POST /socket.io/?EIO=4&transport=polling&t=PjFF9U7&sid=AQx6EsMa7MOxLJXtAAAA HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 07:22:09] "GET /socket.io/?EIO=4&transport=polling&t=PjFF9U9&sid=AQx6EsMa7MOxLJXtAAAA HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 07:22:10] "POST /socket.io/?EIO=4&transport=polling&t=PjFF9cO&sid=AQx6EsMa7MOxLJXtAAAA HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Dec/2025 07:22:10] "GET /socket.io/?EIO=4&transport=polling&t=PjF